In [1]:
import numpy as np

WIDTH = 395
HEIGHT = 860

train_data = np.load('E:/project data/training_data.npy',allow_pickle=True)


In [2]:

import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = 
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

In [3]:
import cv2
X=np.zeros([13000,150,300,3], dtype=np.uint16)
test_x=np.zeros([4000,150,300,3], dtype=np.uint16)
for i in range(len(train_data)):
    if i < 13000:
        a=train_data[i,0][80:475,50:910]
        a=cv2.cvtColor(a,cv2.COLOR_GRAY2RGB)
        a=cv2.resize(a,(300,150))
        X[i]=a
    else:
        a=train_data[i,0][80:475,50:910]
        a=cv2.cvtColor(a,cv2.COLOR_GRAY2RGB)
        a=cv2.resize(a,(300,150))
        test_x[i-13000]=a

In [4]:
import pandas as pd

val=pd.read_csv('C:/Users/Ant Pc/GitHub/AI-in-Mfg---Automated-Driving/Aaftab/choice.csv')
print(val.head())
Y=val.iloc[0:13000,0]
test_y=val.iloc[13000:,0]

   0
0  0
1  0
2  0
3  0
4  0


In [5]:
print(X.shape)
print(test_x.shape)
print(Y.shape)
print(test_y.shape)

(13000, 150, 300, 3)
(4000, 150, 300, 3)
(13000,)
(4000,)


In [6]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

model = Sequential()
model.add(Conv2D(input_shape=(150,300,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=10, activation="softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 300, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 300, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 150, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 150, 128)      73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 150, 128)      147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 75, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 37, 75, 256)       2

In [7]:
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
#model = keras.models.load_model('C:/Users/Ant Pc/GitHub/AI-in-Mfg---Automated-Driving/Aaftab/keras.h5')

from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

In [8]:
from sklearn.utils.multiclass import unique_labels
from keras.utils import to_categorical

#Since we have 10 classes we should expect the shape[1] of y_train,y_val and y_test to change from 1 to 10
print(Y.shape)
Y=to_categorical(Y)
test_y=to_categorical(test_y)

X=X.reshape([13000,150,300,3])
test_x=test_x.reshape([4000,150,300,3])




(13000,)


In [9]:
#Verifying the dimension after one hot encoding
print(X.shape)
print(test_x.shape)
print(Y.shape)
print(test_y.shape)

(13000, 150, 300, 3)
(4000, 150, 300, 3)
(13000, 10)
(4000, 10)


In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1 )

val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1)


In [11]:
train_generator.fit(X)
val_generator.fit(test_x)

In [12]:
batch_size= 2
epochs=100
learn_rate=.1

In [13]:
h=model.fit(train_generator.flow(X,Y, batch_size=batch_size), epochs = epochs, steps_per_epoch = X.shape[0]//batch_size, validation_data = val_generator.flow(test_x, test_y, batch_size=batch_size),verbose=1,callbacks=[checkpoint,early])


Epoch 1/100
6500/6500 [==============================] - 315s 48ms/step - loss: 2.4755 - accuracy: 0.5810 - val_loss: 1.2526 - val_accuracy: 0.5907
Epoch 2/100
6500/6500 [==============================] - 317s 49ms/step - loss: 1.2532 - accuracy: 0.5822 - val_loss: 1.2868 - val_accuracy: 0.5907
Epoch 3/100
6500/6500 [==============================] - 304s 47ms/step - loss: 1.2519 - accuracy: 0.5822 - val_loss: 1.2527 - val_accuracy: 0.5907
Epoch 4/100
6500/6500 [==============================] - 304s 47ms/step - loss: 1.2489 - accuracy: 0.5822 - val_loss: 1.2536 - val_accuracy: 0.5907
Epoch 5/100
6500/6500 [==============================] - 304s 47ms/step - loss: 1.2486 - accuracy: 0.5822 - val_loss: 1.2502 - val_accuracy: 0.5907
Epoch 6/100
6500/6500 [==============================] - 304s 47ms/step - loss: 1.2475 - accuracy: 0.5822 - val_loss: 1.2508 - val_accuracy: 0.5907
Epoch 7/100
6500/6500 [==============================] - 302s 47ms/step - loss: 1.2468 - accuracy: 0.5822 - val_

KeyboardInterrupt: 

In [ ]:
h.history

In [ ]:
model.save('keras.h5')

In [ ]:
import numpy as np
from grabscreen import grab_screen
import cv2
import time
from directkeys import PressKey,ReleaseKey, W, A, S, D
from getkeys import key_check

import random
t_time = 0.09

def straight():
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(D)
    ReleaseKey(S)

def left():
    if random.randrange(0,3) == 1:
        PressKey(W)
    else:
        ReleaseKey(W)
    PressKey(A)
    ReleaseKey(S)
    ReleaseKey(D)
    #ReleaseKey(S)

def right():
    if random.randrange(0,3) == 1:
        PressKey(W)
    else:
        ReleaseKey(W)
    PressKey(D)
    ReleaseKey(A)
    ReleaseKey(S)
    
def reverse():
    PressKey(S)
    ReleaseKey(A)
    ReleaseKey(W)
    ReleaseKey(D)


def forward_left():
    PressKey(W)
    PressKey(A)
    ReleaseKey(D)
    ReleaseKey(S)
    
    
def forward_right():
    PressKey(W)
    PressKey(D)
    ReleaseKey(A)
    ReleaseKey(S)

    
def reverse_left():
    PressKey(S)
    PressKey(A)
    ReleaseKey(W)
    ReleaseKey(D)

    
def reverse_right():
    PressKey(S)
    PressKey(D)
    ReleaseKey(W)
    ReleaseKey(A)
    
def no_keys():

    if random.randrange(0,3) == 1:
        PressKey(W)
    else:
        ReleaseKey(W)
    ReleaseKey(A)
    ReleaseKey(S)
    ReleaseKey(D)

def main():
    last_time = time.time()
    for i in list(range(4))[::-1]:
        print(i+1)
        time.sleep(1)

    paused = False
    while(True):
        
        if not paused:
            # 800x600 windowed mode
            #screen =  np.array(ImageGrab.grab(bbox=(0,40,800,640)))
            screen = grab_screen(region=(0,40,3840,2160))
            last_time = time.time()
            screen = cv2.cvtColor(screen, cv2.COLOR_BGR2GRAY)
            screen = cv2.resize(screen, (0,0), fx=0.25, fy=0.25)
            screen=cv2.cvtColor(screen,cv2.COLOR_GRAY2RGB)
            screen=screen[80:475,50:910]
            screen = cv2.resize(screen, (150,300))

            prediction = model.predict([screen.reshape(1,150,300,3)])[0]
            

            mode_choice = np.argmax(prediction)
            print(prediction)
            if mode_choice == 6:
                straight()
                choice_picked = 'straight'
                
            elif mode_choice == 8:
                reverse()
                choice_picked = 'reverse'
                
            elif mode_choice == 7:
                left()
                choice_picked = 'left'
            elif mode_choice == 9:
                right()
                choice_picked = 'right'
            elif mode_choice == 0:
                forward_left()
                choice_picked = 'forward+left'
            elif mode_choice == 2:
                forward_right()
                choice_picked = 'forward+right'
            elif mode_choice == 3:
                reverse_left()
                choice_picked = 'reverse+left'
            elif mode_choice == 5:
                reverse_right()
                choice_picked = 'reverse+right'
            else:
                no_keys()
                print('no')
                choice_picked = 'nokeys'

        keys = key_check()
        print(choice_picked)
        # p pauses game and can get annoying.
        if 'T' in keys:
            if paused:
                paused = False
                time.sleep(1)
            else:
                paused = True
                ReleaseKey(A)
                ReleaseKey(W)
                ReleaseKey(D)
                time.sleep(1)



In [ ]:
main()